<a href="https://colab.research.google.com/github/sanika-mhadgut/NLP/blob/master/QnA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Name : Sanika Mhadgut
#Branch : Btech Data Science Sem 6
#Roll No: J031

In [13]:
from tqdm import tqdm
import json
import pandas as pd 
import nltk
import re
from nltk.corpus import stopwords
from nltk import word_tokenize 
from string import punctuation,digits
nltk.download('stopwords')
from gensim.models import Word2Vec
from gensim import models
from gensim.test.utils import get_tmpfile, common_texts

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Word2vec is one of the most popular technique to learn word embeddings using a two-layer neural network. Its input is a text corpus and its output is a set of vectors. Word embedding via word2vec can make natural language computer-readable, then further implementation of mathematical operations on words can be used to detect their similarities. A well-trained set of word vectors will place similar words close to each other in that space.

In [15]:
import gensim
from gensim.models import Word2Vec
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [1]:
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2020-02-22 16:18:22--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M   140MB/s    in 0.3s    

2020-02-22 16:18:27 (140 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [0]:
import json
import pandas as pd
df= pd.read_json('/content/train-v2.0.json')

In [3]:
df

,version,data
0,v2.0,"{'title': 'Beyoncé', 'paragraphs': [{'qas': [{..."
1,v2.0,"{'title': 'Frédéric_Chopin', 'paragraphs': [{'..."
2,v2.0,{'title': 'Sino-Tibetan_relations_during_the_M...
3,v2.0,"{'title': 'IPod', 'paragraphs': [{'qas': [{'qu..."
4,v2.0,{'title': 'The_Legend_of_Zelda:_Twilight_Princ...
...,...,...
437,v2.0,"{'title': 'Infection', 'paragraphs': [{'qas': ..."
438,v2.0,"{'title': 'Hunting', 'paragraphs': [{'qas': [{..."
439,v2.0,"{'title': 'Kathmandu', 'paragraphs': [{'qas': ..."
440,v2.0,"{'title': 'Myocardial_infarction', 'paragraphs..."


In [0]:
df=df.data

In [0]:
with open("/content/train-v2.0.json", "r") as f:
    data = json.loads(f.read())

questions, answers = [], []

for i in range(len(data["data"])):
    for j in range(len(data["data"][i]["paragraphs"])):
        for k in range(len(data["data"][i]["paragraphs"][j]["qas"])):
            q = data["data"][i]["paragraphs"][j]["qas"][k]["question"]
            try: 
                a = data["data"][i]["paragraphs"][j]["qas"][k]["answers"][0]["text"]
            except IndexError:
                a = "None"

            questions.append(q)
            answers.append(a)

d = {
    "Questions": questions,
    "Answers": answers
}

In [0]:
Data = pd.DataFrame(d)

In [9]:
Data

,Questions,Answers
0,When did Beyonce start becoming popular?,in the late 1990s
1,What areas did Beyonce compete in when she was...,singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,2003
3,In what city and state did Beyonce grow up?,"Houston, Texas"
4,In which decade did Beyonce become famous?,late 1990s
...,...,...
130314,Physics has broadly agreed on the definition o...,None
130315,Who coined the term partonic matter?,None
130316,What is another name for anti-matter?,None
130317,Matter usually does not need to be used in con...,None


In [0]:
import json
import numpy as np
file = json.loads(open('/content/train-v2.0.json').read())
record_path = ['data','paragraphs','qas','answers']
js = pd.io.json.json_normalize(file ,record_path)
m = pd.io.json.json_normalize(file, record_path[:-1] )
r = pd.io.json.json_normalize(file,record_path[:-2])
idx = np.repeat(r['context'].values, r.qas.str.len())
ndx  = np.repeat(m['id'].values,m['answers'].str.len())
m['context'] = idx
js['q_idx'] = ndx
main =pd.concat([m[['id','question','context','answers']].set_index('id'),js.set_index('q_idx')],1,sort=False).reset_index()
main['c_id'] = main['context'].factorize()[0]

In [11]:
main.head(5)

,index,question,context,answers,text,answer_start,c_id
0,56be85543aeaaa14008c9063,When did Beyonce start becoming popular?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"[{'text': 'in the late 1990s', 'answer_start':...",in the late 1990s,269.0,0
1,56be85543aeaaa14008c9065,What areas did Beyonce compete in when she was...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"[{'text': 'singing and dancing', 'answer_start...",singing and dancing,207.0,0
2,56be85543aeaaa14008c9066,When did Beyonce leave Destiny's Child and bec...,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"[{'text': '2003', 'answer_start': 526}]",2003,526.0,0
3,56bf6b0f3aeaaa14008c9601,In what city and state did Beyonce grow up?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"[{'text': 'Houston, Texas', 'answer_start': 166}]","Houston, Texas",166.0,0
4,56bf6b0f3aeaaa14008c9602,In which decade did Beyonce become famous?,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,"[{'text': 'late 1990s', 'answer_start': 276}]",late 1990s,276.0,0


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
df1=TfidfVectorizer(stop_words='english',max_features=7000)
df1_val=df1.fit_transform(main['question'])
final=pd.DataFrame(df1_val.toarray(),columns=df1.get_feature_names())
final.head(2)





,000,10,100,1000,101,10th,11,1100,11th,12,1200,122nd,12th,13,13th,14,1400,14th,15,1500,15th,16,1600,16th,17,1700,17th,18,1800,1800s,1810,1815,1820,1830,1837,1838,1839,1842,1848,1850,...,wwii,xbox,xi,xiang,xii,xxiii,yacht,yale,yangtze,yard,yards,yaroslav,year,yearly,years,yellow,yeltsin,yiddish,yield,yoga,yongle,york,yorkers,young,younger,youngest,youth,youtube,yuan,yugoslavia,zealand,zelda,zeppelin,zero,zhang,zhejiang,zinc,zone,zones,zoo
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [0]:
remove_digits = str.maketrans('', '', digits)
stop_words = set(stopwords.words('english'))

def tokens(str):
  str = str.lower()
  str = str.translate(remove_digits)
  str = re.sub(r'[^\w\s]',' ',str)
  token = word_tokenize(str)
  words = [word for word in token if word not in stop_words and word not in punctuation]
  return words

Euclidian similarity cannot work well for the high-dimensional word vectors. This is because Euclidian similarity will increase as the number of dimensions increases, even if the word embedding stands for different meanings. Alternatively, we can use cosine similarity to measure the similarity between two vectors. Mathematically, it measures the cosine of the angle between two vectors projected in a multi-dimensional space.

In [16]:
df_final=main.question.apply(tokens)
df_final.shape
def clean(text):
  text=text.lower()
  remove_digits = str.maketrans('', '', digits)
  text = text.translate(remove_digits)
  text = re.sub(r'[^\w\s]',' ',text)
  tokenized_text=word_tokenize(text)
  return tokenized_text
df_final= main.question.apply(clean)
df_final=df_final.tolist()


130319

In [0]:
word2vec=Word2Vec(list(df_final),size=100,min_count=1,sg=1,workers=5,window=3)
s1='What is the question'
s2='This is a Answer'
sim=word2vec.wv.similarity(s1.lower().split(),s2.lower().split())

size: The number of dimensions of the embeddings and the default is 100.


window: The maximum distance between a target word and words around the target word. The default window is 3.


min_count: The minimum count of words to consider when training the model; words with occurrence less than this count will be ignored. The default for min_count is 1.


workers: The number of partitions during training and the default workers is 5.


sg: The training algorithm, either CBOW(0) or skip gram(1). The default training algorithm is CBOW.